In [8]:
!sudo apt-get install git-lfs -y

!git clone https://github.com/amazon-science/esci-data.git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.
Cloning into 'esci-data'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 124 (delta 49), reused 48 (delta 24), pack-reused 36
Receiving objects: 100% (124/124), 390.33 KiB | 2.49 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Filtering content: 100% (2/2), 1.08 GiB | 70.45 MiB/s, done.


In [2]:
!ls esci-data/shopping_queries_dataset

shopping_queries_dataset_examples.parquet  shopping_queries_dataset_sources.csv
shopping_queries_dataset_products.parquet


In [92]:
!pip install langdetect  pandarallel


  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16677 sha256=f656014dbbb225e5928e1d39a4771588dcc0905769f4f8f163281ab8604091d7
  Stored in directory: /root/.cache/pip/wheels/68/bc/40/9e885c7a0454951b6d9b719f0f5ec8a542d3a3df7415acf4cb
Successfully built pandarallel


In [93]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [212]:
import pandas as pd
from langdetect import detect
import string

In [10]:
!ls -lah esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet

-rw-r--r-- 1 root root 49M Nov 19 15:55 esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet


Load examples, products and sources


In [11]:
df_examples = pd.read_parquet('esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet')
df_sources = pd.read_csv("esci-data/shopping_queries_dataset/shopping_queries_dataset_sources.csv")
# df_dataset.head()

In [26]:
df_sources

,query_id,source
0,0,other
1,1,negations
2,2,negations
3,3,negations
4,4,behavioral
...,...,...
130647,130647,other
130648,130648,other
130649,130649,other
130650,130650,other


In [41]:
df_sources['source'].value_counts()

other            113656
negations          6964
parse_pattern      6083
behavioral         3780
nlqec               169
Name: source, dtype: int64

Merge examples with products


In [12]:
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)



Merge examples with query type


In [272]:
# def isEnglish(s):
#     try:
#         s.encode(encoding='utf-8').decode('ascii')
#     except UnicodeDecodeError:
#         return False
#     else:
#         return True
    
    
def is_english(s):
    try:
        return detect(s) == 'en'
    except Exception as e:
        print(( s,e) ) 
        return False
    return False

def is_english(text):
    try:
        text = text.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False

    allowed_characters = set(["$", "!", '"', "-", '#',",",".", "%", "'", '’', "’", ")" ,"(","”",'’','$'])
    try:
        return all(c.isalnum() or c.isspace() or c in allowed_characters or c in string.punctuation for c in text)
    except:
        return False

In [273]:
string.punctuation.split(' ')

['!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']

In [274]:
df_sources

,query_id,source
0,0,other
1,1,negations
2,2,negations
3,3,negations
4,4,behavioral
...,...,...
130647,130647,other
130648,130648,other
130649,130649,other
130650,130650,other


In [275]:
#df_examples_products_query['is_english'] =  df_examples_products_query['query'].apply(isEnglish)

In [276]:
#detect("hi

In [277]:
df_query_lang = df_examples_products[['query','query_id']].drop_duplicates()

In [278]:
df_query_lang

,query,query_id
0,revent 80 cfm,0
16,!awnmower tires without rims,1
32,!qscreen fence without holes,2
70,!solid camiseta sin manga,3
109,"""vitamina c""",4
...,...,...
2621208,ﾎﾙｽﾀｰ,130647
2621224,ﾏｼﾞｯｸﾘﾝ,130648
2621240,ﾒｽﾃｨﾝ,130649
2621256,ﾚﾃﾞｨｰｽ水着,130650


In [279]:
df_query_lang['is_english'] = df_query_lang['query'].parallel_apply(is_english)


In [283]:
df_query_lang['is_english'].value_counts()

True     111890
False     18762
Name: is_english, dtype: int64

In [280]:
df_examples_products_query = pd.merge(
    df_examples_products,
    df_query_lang,
    how='inner',
    on=['query_id','query']
)


In [281]:
list(df_query_lang [df_query_lang['is_english']==False].head(10_000)['query'])

['$12 label maker that’s not a cheap one',
 '$60 ps4 that’s not gonna be on amazon',
 '&#34;tortillas without interesterified soybean oil”',
 '&ハニー シャンプー',
 "'74ジェイルブレイク\u3000ac/dc",
 '(ficción sin límites)',
 '(if you’re not in it for love) i’m outta here!',
 '(i’m not your) steppin’ stone',
 '(metros) bandera españa sin escudo',
 '(éxito y productividad sin límites',
 '(マックバースト） mach burst 軽い コンフォート 穴開き スポーツ サドル 痛くない (ブラック×ブラック)',
 '(知っていることはあまりないけれど)家族です',
 '+jamón sin azucar',
 ', auriculares inalámbricos sin tapón para los oídos',
 ', champú sin sal salerm',
 ', cuentakilómetros de bicicleta inalámbrico sin móvil',
 ', filtro sin fondo de café prime',
 ', máquina de depilación sin dolor',
 ', tableta para dibujar y borrar electrónica sin gastar papel',
 ',,, _ biscotes tostados sin sal y sin azúcar',
 ',filtro sin fondo de café prime',
 ',sujetador sin aro talla pequeña',
 '- [ ] i’m not myself these days josh kilmer-purcell',
 '- butaca de diseño de terciopelo sin reposabrazos',


In [282]:
is_english("$60 ps4 that’s not gonna be on amazon")

False

In [189]:
is_english("mom life")

True

In [190]:
detect("mom life")

'no'

In [109]:
df_examples_products_query\
.pipe(lambda x: x[x.product_locale =="us"]) \
.replace({"esci_label": {'E':'Exact' ,'S':'Substitute', 'C':'Complement', 'I':'Irrelevant' } })\
.rename (columns = {'esci_label':'relevance_label','source':'query_type'} ) \
.drop(columns = ['example_id','query_id','product_locale','small_version','large_version','split'] )

,query,product_id,relevance_label,product_title,product_description,product_bullet_point,product_brand,product_color,is_english
0,revent 80 cfm,B000MOO21W,Irrelevant,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,False
1,revent 80 cfm,B07X3Y6B1V,Exact,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,False
2,revent 80 cfm,B07WDM7MQQ,Exact,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,False
3,revent 80 cfm,B07RH6Z8KW,Exact,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,False
4,revent 80 cfm,B07QJ7WYFQ,Exact,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White,False
...,...,...,...,...,...,...,...,...,...
2618565,자전거트레일러,B06XTZYJL3,Irrelevant,Nilight - ZH003 20Inch 126W Spot Flood Combo L...,None,Super bright beam: High intensity 3W LED chips...,Nilight,None,False
2618566,자전거트레일러,B010LLGWL8,Exact,"Burley Honey Bee, 2 Seat Kids Bike Trailer & S...",None,Sport Type: Cycling,Burley Design,Red,False
2618567,자전거트레일러,B010LLGWKE,Exact,"BURLEY Design Bee, 2 Seat, Lightweight, Kids B...",None,sport type: Cycling,Burley Design,Yellow,False
2618568,자전거트레일러,B003CUBPUY,Irrelevant,"Bell 20-Inch Universal Inner Tube, Width Fit R...",None,Mold cured rubber for consistent side wall–pre...,BELL,"20""x1.75-2.25"" Schrader",False


In [16]:
len(df_examples_products['product_id'].unique())

1802772

In [24]:
df_mask = df_examples_products['product_locale'].isin(['us'])

len( df_examples_products[df_mask]['product_id'].unique()) 

1215851

In [25]:
df_examples_products[df_mask]['product_id'].unique()

array(['B000MOO21W', 'B07X3Y6B1V', 'B07WDM7MQQ', ..., 'B00QN6LC9I',
       'B07Z44KPHS', 'B003CUBPUY'], dtype=object)

In [ ]:
df_dataset['source'].value_counts()

In [ ]:
df_product = pd.read_parquet("esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet")
df_product.head()

In [ ]:
len(df_product)

In [ ]:
df_product['product_brand'].nunique()

In [ ]:
df_product['product_locale'].value_counts()

In [ ]:
df_product_en = df_product[df_product['product_locale']=="us"]

In [ ]:
df_product_en.iloc[500].to_dict()

In [ ]:
df_product_en.iloc[600].to_dict()

In [7]:
df_examples = pd.read_parquet("esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet")
df_examples.head()

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:
df_examples['esci_label'].value_counts()

In [ ]:
len(df_examples)

In [ ]:
df_examples['query'].nunique()

In [ ]:
df_uniqeu_quereis = df_examples['query'].drop_duplicates()

In [ ]:
list ( df_uniqeu_quereis.head(100) )

In [ ]:
df_examples['product_id'].nunique()

In [ ]:
df_examples [df_examples['product_id']=="B000BQWKCY"]

In [ ]:
df_examples['query'].unique()

In [ ]:
df_examples [df_examples['query']=="$25 ceiling fan without lights"]